# About Converting from SAS to Dataframes
Each dataset is examined and concatenated in a way that makes the most sense for that set. Further, column types
are set for more efficient storage.

Currently, this does not convert all dataset, but does provide tools and a template to convert any datasets not yet converted by this notebook.

## Merging visit datafiles

145 different data files is crazy to manage for folks who are looking for correlations across datasets collected from the OAI. It has also led to a massive number of variables being defined: 11,000+. This is over 3x the actually number of unique variables measured across all visits. 

Given that several files are same/similar data collected across visits, this notebook defines tools to collect all that data into single files with columns to mark which visit a variable corresponds to. This greatly reduces the variable namespace. Whereas you may originally have:

file 1:
```
ID V00FOO
1   30.5
2   24.7
```

file 2:
```
ID V01FOO
1   31.9
2   27.3
```
This merges it into a single dataframe:
```
ID Visit FOO
1   V00  30.5
2   V00  24.7
1   V01  31.9
2   V01  27.3
```
## Port data types / Set efficient storage

In storage, SAS only stores values as floats or strings. This is extremely inefficient. When the pyreadstat library reads the data it has some rudimentary detection of column types. Still, most OAI columns endup as dtype object. Thus column types still need to be examined and cast to the most efficient data type.

Also, SAS allows for multiple user defined data markers for any column of data. This takes two forms. The first is similar to Panda's categorical types. This can be stored efficiently in Pandas. The second allows for multiple types of missing data (where Pandas only has NA or NaN). This causes most columns to be of mixed types in Pandas (float and str).

### Preserving SAS missing values in Pandas
By allowing for user defined missing values, SAS allows you to either treat all missing values the same or leverage the fact that not all values are missing for the same reason. Neither Python Pandas (or even R's dataframes) allow this as directly. Rather than throw this information away, for any dataset that includes missing values, two dataframes will be created. The first contains all the data with NaN or NA in place of all missing values. The second will be a dataframe with NaN in place of all values, but containing the full missing value labels at the same indices they existed in the original data. This allows ignoring missing values as the default case, but when needed, an NA in the data dataframe can trigger a check for a missing value in the missing value dataframe.

e.g. data
```
ID Visit FOO   BAR  BAZ
1   V00  30.5   5   0.1
2   V00  NaN    7   0.5 
1   V01  31.9   1   0.6
2   V01  27.3   NA  1.2
```
e.g. missing values/"shadow" dataframe
```
ID Visit FOO   BAR
1   V00  30.5   5
2   V00  NaN    7
1   V01  31.9   1
2   V01  27.3   NA
```

Note that columns with no missing values do not get copied to the shadow dataframe (with the exception of ID and Visit for indexing purposes).

### TODO

Currently 5 files aren't handled yet. In each, the column naming format doesn't use visit prefixes:
* Biospec_fnih_joco_demographics
* biospec_fnih_joco_assays
* kmri_poma_incoa_moaks_bicl
* kmri_poma_tkr_chondrometrics
* kmri_poma_tkr_moaks_bicl
    
 A separate create_df() function needs to be made to handle these cases.


More can be done here. For example:

* Improve suggest_conversion():
    * add the ability to detect boolean columns
    * check for possible categoricals in numeric columns with few unique values
    * handle columns of numeric strings (double check they have a leading 0 or other reason to be a string)

* Sanity checks:
    * in SAS string missing values are a blank, did pyreadstat capture this? string cols seem to have empty strings
    
* Other improvements:
    * Add method to summarize and sanity check the missing_df shadow dataframes

# Conversion Tools
Script to create a global dictionary of metadata needed to convert to efficient datatypes, along with functions to examine and convert the SAS data into Pandas dataframss.

## Imports / Definitions

In [ ]:
from copy import deepcopy
import datetime
import math
import os

import numpy as np
import pandas as pd
import pandas.api.types as pdtypes
import pickle
import pyreadstat
import re
from string import digits
from tqdm import tqdm

# Setup 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>.output_result { max-width:95% !important; }</style>"))

In [ ]:
# Constants
data_dir = '../data/structured_data/'
visits = {'P02':'IEI', 'P01':'SV', 'V00':'EV', 'V01':'12m', 'V02':'18m', 'V03':'24m', 'V04':'30m', 'V05':'36m',
          'V06':'48m', 'V07':'60m', 'V08':'72m', 'V09':'84m', 'V10':'96m', 'V11':'108m', 'V99':"Outcomes"}
visit_prefixes = set(visits.keys())

meta_vars = [ 'column_labels',
 'column_names',
 'column_names_to_labels',
 'file_encoding',
 'file_format',
 'file_label',
 'missing_ranges',
 'missing_user_values',
 'notes',
 'number_columns',
 'number_rows',
 'original_variable_types',
 'readstat_variable_types',
 'table_name',
 'value_labels',
 'variable_alignment',
 'variable_display_width',
 'variable_measure',
 'variable_storage_width',
 'variable_to_label',
 'variable_value_labels']

metadata_dict_names = ['column_names_to_labels', 'original_variable_types', 'readstat_variable_types',
                       'value_labels', 'variable_alignment', 'variable_display_width',
                       'variable_measure', 'variable_storage_width', 'variable_to_label', 'variable_value_labels']

default_missing_value_codes = {
    ' ': '.: Missing Form/Incomplete Workbook',
    'A': '.A: Not Expected',
    'B': '.B: Low/Below Range',
    'C': '.C: Cannot Do/Attempted: unable to complete',
    'D': '.D: Don’t Know/Unknown/Uncertain',
    'E': '.E: Non-Exposed Control',
    'F': '.F: Not done, phone contact',
    'G': '.G: Unreleased high value',
    'H': '.H: High/Above range',
    'I': '.I: Inadequate data',
    'K': '.K: Cannot do/not attempted, unable',
    'L': '.L: Permanently Lost',
    'M': '.M: Missing',
    'N': '.N: Not Required/Not edited',
    'O': '.O: Not done, other reason',
    'P': '.P: Prosthetic',
    'R': '.R: Refused',
    'S': '.S: Unreleased low value',
    'T': '.T: Technical problems',
    'U': '.U: Unable to examine',
    'V': '.V: Missed visit',
    'W': '.W: Impossible value'
}
default_missing_val_tokens = set(default_missing_value_codes.keys())

## Look at the filesets

In [ ]:
# All SAS files
all_files = os.listdir(data_dir)
all_files = [x for x in all_files if '.sas7bdat' in x]
all_files.remove('sageancillarystudy_formats.sas7bdat') ## At a binary level this seems like another CPORT file. WTF?
all_files.sort()

# How many files are there?
print('File cnt: ' + str(len(all_files)))

# How many sets?
# Drop extensions and then drop visit suffixes
tmp = set([f.translate(f.maketrans('', '', digits)) for f in [f.removesuffix('.sas7bdat') for f in all_files]])
print('File set cnt: ' + str(len(tmp)))

## Utility Functions

In [ ]:
#   Given a common filename prefix, return the sorted list of sas7bdat files starting with that prefix
# e.g. 'foo' -> ['foo01.sas7bdat', 'foo02.sas7bdat']
def get_data_file_names(prefix):
    file_list = [x for x in all_files if x.startswith(prefix)]
    file_list.sort()
    return file_list


#   Clean visit prefixes
# e.g. ['V01FOO', 'v02BAR'] -> ['FOO', 'BAR']
def remove_visit_prefixes(str_list):
    return [s[3:] if re.match("^[vVpP]\d\d\D\S*", s) else s for s in str_list]


#   Return a list of all unique prefixes
# e.g. ['V01FOO', 'V02BAR'] -> ['V01', 'V02']
def collect_prefixes(str_list):
    return list({s[:3] for s in str_list if re.match("^[vVpP]\d\d\D\S*", s) })

#   Get value_labels from catalog file
# The dictionaries that define user-defined types are stored in '.sas7bcat' files
def get_value_labels(filepath):
    _, data_catalog = pyreadstat.read_sas7bcat(filepath)
    return data_catalog.value_labels

#    Debug funct to see which files a given var name exists and the datatype in each file
# e.g. FOO -> bar01.sas7bcat V01FOO string 3 $
def show_sources_types(var):
    for file, meta in files_meta.items():
        for v in meta.column_names:
            if v.endswith(var):
                print(file + ' ' + v + ' ' + meta.readstat_variable_types[v] + ' ' + str(meta.variable_storage_width[v]) + ' ' + meta.variable_to_label.get(v, ''))

## Creating a Global Metadata Dictionary
The `.sas7bdat` files contain the raw data, and most metadata.  `.sas7bcat` includes definitions for the user defined types.

This section defines functions for handling this metadata. This primarily reads in all metadata across datasets and creates a single global dictionary of metadata. This dictionary
can then be used to correctly set the data types for any data read in from subsequent `.sas7bdat` files.

We don't care about the entire metadata set returned by pyreadstat, only `['column_names', 'readstat_variable_types', 'variable_storage_width', 'variable_to_label']`
- See the `Exploring Available SAS Metadata` notebook for details on user-defined types and the meanings of various metadata items

In [ ]:
# Grab the metadata across all files
# - Roughly ~1.5 min runtime
files_meta = {}
for filename in all_files:
    _, meta = pyreadstat.read_file_multiprocessing(pyreadstat.read_sas7bdat, data_dir + filename,
                                                         num_processes=6, metadataonly=True)
    files_meta[filename] = meta

In [ ]:
# Normalize names
#  - SAS is case insensitive, to normalize names for Python, all variables from SAS are migrated to uppercase
for file, meta in files_meta.items():
    meta.column_names = [n.upper() for n in meta.column_names]
    for v_name in ['readstat_variable_types', 'variable_storage_width', 'variable_to_label']:
        var = getattr(meta, v_name)
        setattr(meta, v_name, {k.upper(): v for k,v in var.items()})

In [ ]:
# Make master map of original variable names to collapsed names
# - This dict maps names like V01FOO -> FOO
var_name_map = {}
for meta in files_meta.values():
    new = {n: (n[3:] if n[:3] in visit_prefixes else n) for n in meta.column_names}
    var_name_map = {**var_name_map, **new}

In [ ]:
# Building the global metadata dictionary
# global_var_name - {storage_type, storage_width, user_defined_type}
#
# - The collapse of names causes 4 conflicts:
#      name   type  width - [files]
# COHORT
#   V00COHORT double 8 - [Clinical_fnih.sas7bdat, enrollees.sas7bdat]
#   COHORT string 11 - [measinventory.sas7bdat]
# RACE
#   RACE string 1 - [Biospec_fnih_joco_demographics.sas7bdat ]
#   P02RACE double 8 - [Clinical_fnih.sas7bdat, enrollees.sas7bdat, measinventory.sas7bdat]
# PTH
#   Has value_label YNDK for allclinicalXX.sas7bdat, but no type in boneancillarystudy.sas7bdat
# READER
#   Has value_label '$' in most files, but no type in kmri_sq_biclXX.sas7bdat
#
# This code captures the common setting and leaves the outliers for custom treatment later

global_meta = {}
for meta in files_meta.values():
    for col in meta.column_names:
        if col not in ['RACE', 'COHORT']: # Exceptions, see above
            descript = meta.column_names_to_labels.get(col, None)
            storage_type = meta.readstat_variable_types[col]
            storage_width = meta.variable_storage_width[col]
            data_type = meta.variable_to_label.get(col, None)
            if data_type in ['$', 'BEST', 'MMDDYY']: 
                data_type = None
            if global_meta.get(var_name_map[col]) and col not in ['ID', 'VERSION']:
                # storage_type
                assert global_meta[var_name_map[col]]['storage_type'] == storage_type
                # Set to largest storage width seen so far
                # width
                if global_meta[var_name_map[col]]['storage_width'] != storage_width:
                    global_meta[var_name_map[col]]['storage_width'] = max(global_meta[var_name_map[col]]['storage_width'], storage_width)
                # data_type
                assert global_meta[var_name_map[col]]['data_type'] == data_type or not data_type
            else:       
                global_meta[var_name_map[col]] = {'storage_type': storage_type, 'storage_width': storage_width, 'data_type': data_type, 'descript': descript}
global_meta['Visit'] = {'storage_type': None, 'storage_width': None, 'data_type': None, 'descript': 'Which visit this data was collected during'}

### Clean up user defined types (SAS value labels) and add to global metadata

Finish the construction of the global metadata map by adding in value-labels (SAS mechanism for defining categoricals and missing value flags)

The following sanity checks are applied:
* Get rid of all NaNs and double NaNs in dictionaries
* Confirm that if V01FOO & V02FOO collapse into FOO, then the datatypes are truly the same

In [ ]:
# Grab the system-wide map of all value_labels to their user-defined type dict
value_labels = get_value_labels(data_dir + 'formats.sas7bcat')

# Create a list of value_labels associated with a double (those for data stored as strings seem fine)
vl_doubles = []
for name in value_labels.keys():
    for var, meta in global_meta.items():
        if meta['data_type'] == name and meta['storage_type'] == 'double':
            vl_doubles.append(name)
            break

# Clean out NaNs and double NaNs            
for name in vl_doubles:
    value_labels[name] = {('.' if isinstance(k, float) and math.isnan(k) else k):v for k,v in value_labels[name].items()}
    
# Swap out data_type name for actual dictionary from value_labels
# add in CategoricalDtype objects to be reused
for meta in global_meta.values():
    if meta['data_type'] and value_labels.get(meta['data_type']):
        meta['data_type'] = value_labels[meta['data_type']]
        meta['CategoricalDtype'] = pd.CategoricalDtype(meta['data_type'].values())

### Look at the scope of the name collapse

In [ ]:
# How large is the name collapse?
tot_col_set = set()
for meta in files_meta.values():
    tot_col_set.update([n.upper() for n in meta.column_names])
print('Original number of unique variables: ' + str(len(tot_col_set)))
print('Collapsed numer of variables: ' + str(len(set(var_name_map.values()))))

## Create a single dataframe out of multiple datasets
Create a single dataframe for all variables across a given fileset.

This only sets the datatype for some columns.
* ID - set to unsigned int
* Visit, Version - set to Categorical
* columns where all values are SAS user-defined - set Categorical

This means that numeric columns with a mix of missing values and numbers are not automatically converted. Later functions allow you to investigate the data more closely, suggest conversions, and convert the columns as you best see fit.

In [ ]:
def create_df(prefix):
    # read in data from each file and append data to master dataframe
    df_list = []
    for filename in get_data_file_names(prefix):
        tmp_df, _ = pyreadstat.read_file_multiprocessing(pyreadstat.read_sas7bdat, data_dir + filename,
                                                            num_processes=6, user_missing=True)
       
        # Normalize column names to uppercase (SAS is case insensitive and variable names are inconsistent across data files)
        # DON'T drop prefixes here because a some files have data for more than one visit and common 
        # variables would collapse (and Pandas will complain about dup column names with an obscure msg)
        tmp_df = tmp_df.rename(columns={c: c.upper() for c in tmp_df.columns})
        print(filename + '\tVar Cnt: ' + str(len(tmp_df.columns)))
            
        # What visits does this dataframe cover?
        inc_visits = collect_prefixes(tmp_df.columns)
        inc_visits.sort()
        print('Visits: ' + str(inc_visits))
        
        # Process all variables collected in the same visit at the same time
        for visit in inc_visits:
            visit_vars = ['ID', 'VERSION']
            visit_vars.extend([v for v in tmp_df.columns if v.startswith(visit)])
            tmp2_df = tmp_df[visit_vars]
            new_cols = {c: c.removeprefix(visit) for c in visit_vars}  # drop visit prefixes from variable names
            tmp2_df = tmp2_df.rename(columns=new_cols)
            # Categorical values must be set to the master set of all values for each column,
            # otherwise categorical lists don't match when concatenating and revert to strings
            for col in tmp2_df.columns:
                dt = global_meta[col]['data_type']
                if dt:
                    if isinstance(dt, dict):
                        # Need to swap out . for NaN
                        if global_meta[col]['storage_type'] == 'double':
                            tmp2_df[col].fillna('.', inplace=True)
                        tmp2_df[col] = tmp2_df[col].apply(lambda x: dt.get(x, x))
                        tmp2_df[col] = tmp2_df[col].astype(global_meta[col]['CategoricalDtype'])
                    else:
                        print('Unhandled data type: ', col, dt)
            tmp2_df.insert(1, 'Visit', visit)  # Mark which visit these variables are associated with
            # print(visit, tmp2_df.columns)
            df_list.append(tmp2_df)

    master_df = pd.concat(df_list, axis=0)
    master_df['ID'] = pd.to_numeric(master_df['ID'], downcast='unsigned')
    master_df = master_df.astype({col: 'category' for col in ['Visit', 'VERSION']})

    return master_df

## Dataset Inspection and Type Optimization Functions
A set of functions to examine the data being converted and help detect the columns that may need manual inspection before conversion.

create_df() only sets a few column types. These function looks for columns that likely need manual conversion.

In [ ]:
#     Highlights which columns change between files
#
# Across a series of files data00.sas7bdat-data11.sas7bdat columns may be added or
# dropped based on the evolution of questions asked in OAI. Further, capitalization
# may change for the column names. This function highlights those changes.
# 
# Run before create_df() to see what is in the 'sas7bdat' files
def column_uniformity_check(prefix):
    tot_cnt = 0
    col_set = {}
    for filename in get_data_file_names(prefix):
        tmp_df, _ = pyreadstat.read_file_multiprocessing(pyreadstat.read_sas7bdat, data_dir + filename,
                                                         catalog_file=data_dir + 'formats.sas7bcat',
                                                         num_processes=6, user_missing=True)
        tot_cnt += tmp_df.shape[0]
        print('\n' + filename + ': '+ str(tmp_df.shape))
        
        col_names = remove_visit_prefixes(list(tmp_df.columns))
        if not col_set:
            print(col_names)
            col_set = set([c.upper() for c in col_names])
        # display any new or missing elements 
        elif col_set ^ set(col_names):
            # Weed out difference solely from SAS case insensitivity
            upper_col_names = set([c.upper() for c in col_names])
            if not col_set ^ upper_col_names:
                print('Names only differ by case')
            else:
                print(list(col_set ^ set(upper_col_names)))
                col_set = set(upper_col_names)

    print('\nTotal rows: ' + str(tot_cnt))

In [ ]:
#   After create_df() has been called, gather_column_data_stats() looks at the contents of each column and outputs
# its findings in a dataframe. Indexed by the column names in the original dataset, this dataframe allows for
# easy lookup of the column label, numeric/string/na/date value counts, min and max numeric values, the presence
# of decimal places, and lists of unique missing values and strings present.
#
# Using the type data from gather_column_data_stats(), suggest_conversions() can suggest likely column types for each
# column. It outputs a dictionary that can be pasted into a cell (modified as desired) and then serve as input to 
# convert_columns().



#   Examine dataframe and return information about the data present in each column in a column indexed dataframe
# Two dataframes are returned. The first contains data summaries for all colummns that likely still need
# conversion. The second one containing data summaries for all columns already in categorical or uint format
# (presumably from create_df).
def gather_column_data_stats(df):
    conv_list, done_list = [], []

    for col in df.columns:
        label = global_meta[col]['descript']
        col_dtype = df[col].dtype
        na_cnt = df[col].isna().sum()

        # Type matches already converted column types
        if col_dtype in ['category', np.uint32]:
            done_list.append({'col': col, 'label': label, 'type': col_dtype, 'na_cnt': na_cnt})

        # 100% numeric, check for type ((un)signed int, float)
        elif col_dtype == float:
            num_type, num_cnt, uniq_num, max_num, min_num = examine_numeric_vals(df[col])
            add_entry(conv_list, col, label, num_type=num_type,
                      uniq_num=uniq_num, max_num=max_num, min_num=min_num, num_cnt=num_cnt, na_cnt=na_cnt)

        # object columns can be mixed types, gather stats on all type present
        elif col_dtype == object:
            uniq_strs, str_list, mv_list, numeric_str = None, None, None, None
            num_type, uniq_num, max_num, min_num = None, None, None, None
            mv_cnt, str_cnt, num_cnt, date_cnt, na_cnt = 0, 0, 0, 0, 0

            
            # look at data of each type in column
            col_types = list(df[col].apply(type).unique())   
            for data_type in col_types:
                col_subset = df[col][df[col].apply(lambda x: isinstance(x, data_type))] 
                if data_type == float:
                    num_type, num_cnt, uniq_num, max_num, min_num = examine_numeric_vals(col_subset)
                elif data_type == str:
                    str_cnt, uniq_strs, str_list, mv_cnt, mv_list, numeric_str = examine_str_vals(col_subset)
                elif data_type == datetime.date:
                    date_cnt = col_subset.shape[0]
                else:  # Unexpected datatype
                    print('{} contained unexpected data type: {}'.format(col, data_type))
                    
            # Add stats to entry for this data column
            add_entry(conv_list, col, label, uniq_strs=uniq_strs, str_list=str_list, mv_cnt=mv_cnt, mv_list=mv_list,
                      numeric_str=numeric_str, num_type=num_type, uniq_num=uniq_num, max_num=max_num, min_num=min_num, 
                      str_cnt=str_cnt, num_cnt=num_cnt, date_cnt=date_cnt, na_cnt=na_cnt)
        else:
            print('{} contained unexpected data type: {}'.format(col, data_type))
        
    return pd.DataFrame(conv_list).set_index('col'), pd.DataFrame(done_list).set_index('col')


#   Using the output of gather_column_data_stats() display a dictionary suggesting what Pandas types each
# column should be converted to.
def suggest_conversions(df):
    print('targets = {')
    # Dates
    count_cols = ['str_cnt', 'num_cnt', 'date_cnt', 'na_cnt']
    subset = df[count_cols].apply(lambda x: x > 0)
    dates = df[~subset.str_cnt & ~subset.num_cnt & subset.date_cnt].index.to_list()
    dates.extend(df[subset.str_cnt & ~subset.num_cnt & subset.date_cnt & (df.uniq_strs == 0)].index.to_list())
    if dates:
        print('# Columns with only dates, missing, and NA values')
        print("'date': {},\n".format(dates))
    
    # Numeric
    numeric = df[subset.num_cnt & ~subset.date_cnt & (df.uniq_strs == 0)]
    unsigned = numeric[numeric.num_type == 'unsigned'].index.to_list()
    if unsigned:
        print('# Columns with only unsigned ints, missing, and NA values')
        print("'unsigned': {},\n".format(unsigned))

    signed = numeric[numeric.num_type == 'signed'].index.to_list()
    if signed:
        print('# Columns with only signed ints, missing, and NA values')
        print("'signed': {},\n".format(signed))

    floats = numeric[numeric.num_type == 'float'].index.to_list()
    if floats:
        print('# Columns with only floats, missing, and NA values')
        print("'float': {},\n".format(floats))

    # Strings
    strings = df[subset.str_cnt & ~subset.num_cnt & ~subset.date_cnt].index.to_list()
    if strings:
        print('# Columns with only strings, missing, and NA values')
        print("'cat': {},\n".format(strings))
    print('}\n')
    
    print('\nHandled columns: {}'.format(len(dates) + len(unsigned) + len(signed) + len(floats) + len(strings)))
    
    missing_cols = set(df.index.to_list()) - set(dates+unsigned+signed+floats+strings)
    if missing_cols:
        print('Unhandled columns: {}'.format(missing_cols))
        

#   Given a dictionary mapping types to lists of columns to convert to that type, along with the type information
# information dataframe, and the data itself, return a dataframe with the corresponding columns converted as well
# as a matching "shadow" dataframe of missing values.
#
# If no missing values are present anywhere in the dataset, an empty dataframe is returned
def convert_columns(targets, data_stats_df, df):
    # create the shadow datframe for missing values
    all_cols = [col for col_list in targets.values() for col in col_list]
    cols_w_missing = [col for col in all_cols if data_stats_df.loc[col, 'missing_val_cnt'] > 0]
    missing_df = df[cols_w_missing].applymap(lambda x: x if isinstance(x, str) else np.NaN)
    missing_df = missing_df.applymap(lambda x: default_missing_value_codes.get(x, x))
    missing_df = missing_df.astype('category')
    if not missing_df.empty:
        missing_df['ID'] = df['ID']
        missing_df['Visit'] = df['Visit']
        missing_df = missing_df.set_index(['ID', 'Visit'])
        
    for col_type, cols_to_conv in targets.items():
        na = np.NaN
        if col_type in ['unsigned', 'signed']:
            na = pd.NA
        
        # Now that missing values are copied to a shadow dataframe, remove them from the dataframe
        cols_w_missing = [col for col in cols_to_conv if data_stats_df.loc[col, 'missing_val_cnt'] > 0]
        replace_dict = {col: {val: na for val in data_stats_df.loc[col, 'missing_val_list']} for col in cols_w_missing}
        df[cols_w_missing] = df[cols_w_missing].replace(replace_dict)   
        
        if col_type in ['signed', 'unsigned']:
            cols_w_nas = [col for col in cols_to_conv if data_stats_df.loc[col, 'na_cnt'] > 0]
            df[cols_w_nas] = df[cols_w_nas].fillna(pd.NA) # Manual conversions needed due to presence of NaNs
            cols_w_nas = list(set(cols_w_nas + cols_w_missing))
            df[cols_w_nas] = df[cols_w_nas].astype('Int32')  # Int32 allows for pd.NA values
            
            df[cols_to_conv] = df[cols_to_conv].apply(pd.to_numeric, downcast=col_type)

        elif col_type == 'float':
            df[cols_to_conv] = df[cols_to_conv].apply(pd.to_numeric, downcast='float')
        elif col_type == 'cat':
            df[cols_to_conv] = df[cols_to_conv].astype('category')
        elif col_type == 'date':
            df[cols_to_conv] = df[cols_to_conv].astype('datetime64[ns]')
   
    return df.set_index(['ID', 'Visit']), missing_df


# Helper functions

#   Check for numeric type ((un)signed int, float. NaN), min, max, unique cnt 
def examine_numeric_vals(col):
    num_type = 'float'
    min_val, max_val = col.min(), col.max()
    unique_cnt = len(col.unique())

    # Does it really need to be a float?
    if col[~col.isna()].apply(float.is_integer).all():  # ignoring NaNs, are the rest integers?
        num_type = 'unsigned'
        if min_val < 0:
            num_type = 'signed'

    num_cnt = col.shape[0] - col.isna().sum()
    # Are they all NA?
    if num_cnt == 0:
        num_type = 'na'
            
    return num_type, num_cnt, unique_cnt, max_val, min_val


#   Get a list of unique strs, and whether strings are numbers
def examine_str_vals(col):
    missing_vals = set([k for k in default_missing_value_codes.keys()])
    uniques = set(col.unique()) - missing_vals
    missing_vals = missing_vals & set(col.unique())
    unique_cnt = len(uniques)
    
    # Are they all strings written as strings?
    numeric_str = False
    if not pd.to_numeric(col, errors='coerce').isna().any():
        numeric_str = True
    return col.shape[0], len(uniques), uniques, len(missing_vals), missing_vals, numeric_str


#    Add column data to list
# - convenience function to reduce argument count
def add_entry(conv_list, col_name, label, uniq_strs=0, str_list=None, mv_cnt=0, mv_list=None, numeric_str=None,
              num_type=None, uniq_num=None, max_num=None, min_num=None, str_cnt=0, num_cnt=0, date_cnt=0, na_cnt=0):
    conv_list.append({'col': col_name, 'label': label, 'uniq_strs': uniq_strs, 'str_list': str_list, 
                      'missing_val_cnt': mv_cnt, 'missing_val_list': mv_list, 'numeric_str': numeric_str,
                      'num_type': num_type, 'uniq_num': uniq_num, 'max_num': max_num, 'min_num': min_num, 
                      'str_cnt': str_cnt, 'num_cnt': num_cnt, 'date_cnt': date_cnt, 'na_cnt': na_cnt})

In [ ]:
#         Column data stats summary functions - called after gather_column_data_stats()

#   Print a dataframe wide summary of the datatypes found in the analysis
# arguments - data dataframe, data summary dataframe
def data_stats_summary(df, data_stats_df):
    print('Already defined cols: {} \tCols to convert: {}\t Total col cnt: {}'.format(df.shape[1] - data_stats_df.shape[0], data_stats_df.shape[0], df.shape[1]))
    print('\nColumn types to convert:\n{}'.format(column_types_present(data_stats_df)))
    print('\nNumeric types of columns:\n{}'.format(data_stats_df['num_type'].value_counts()))
    print('\nLargest number of unique strings: {}'.format(data_stats_df['uniq_strs'].max()))
    #print('\nHistogram of different NA count sizes:\n{}'.format(data_stats_df['na_cnt'].value_counts()))

    
#    Print a table of the combinations of data types found across columns in the analysis
# arguments - data summary dataframe
def column_types_present(df):
    count_cols = ['str_cnt', 'num_cnt', 'date_cnt', 'na_cnt']
    subset = df[count_cols]
    subset = subset.apply(lambda x: x > 0)
    return subset.groupby(count_cols).size().reset_index().rename(columns={0:'count'})

In [ ]:
# Inspect values and types
                
#   Show string columns data to look for patterns
def show_string_col_stats(stats_df):
    count_cols = ['str_cnt', 'num_cnt', 'date_cnt', 'na_cnt']
    subset = stats_df[count_cols].apply(lambda x: x > 0)
    return stats_df[subset.str_cnt & ~subset.num_cnt & ~subset.date_cnt & subset.na_cnt]                


#    Check the converted dataframe for wrong and suspcious things
#  List which columns aren't categoricals though create_df should have made them so
#  List which columns have NA/NaN values though they weren't expected to
def sanity_check(df):
    # Confirm all categorical columns expected to be categorical are
    for col in df:
        if col not in ['Visit', 'VERSION'] and  global_meta[col]['data_type']:
            if not pdtypes.is_categorical_dtype(df[col]):
                print('Failure to make column categorical: ' + col)
            if df[col].isna().all():
                print('All NaN in categorical col: ' + col)
    if df.select_dtypes(include='object').columns.to_list():
        print('Columns still object type: ', df.select_dtypes(include='object').columns.to_list())         

# Dataset Conversions
Processed in alphabetical order

## Biospec_fnih_joco_demographics
TODO: Not handled yet as column naming format doesn't use visit prefixes

In [ ]:
prefix = 'Biospec_fnih_joco_demographics'
column_uniformity_check(prefix)

## Clinical_fnih

In [ ]:
prefix = 'Clinical_fnih'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only unsigned ints, missing, and NA values
'unsigned': ['AGE'],

# Columns with only floats, missing, and NA values
'float': ['BMI', 'XRJSM', 'XRJSL', 'MCMJSW', 'WOMKP', 'WOMADL'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## acceldatabyday

In [ ]:
prefix = 'acceldatabyday'
column_uniformity_check(prefix)  # Becasue there is more than one dataset file

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)

data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only unsigned ints, missing, and NA values
'unsigned': ['PASTUDYDAY', 'VDAYSEQUENCE', 'PAMONTH', 'DAYMODMINT', 'DAYMODMINF', 'DAYMODMINS', 'DAYVIGMINT', 'DAYVIGMINF', 'DAYVIGMINS', 'DAYMVMINT', 'DAYMVMINF', 'DAYMVMINS', 'DAYCNT', 'DAYLTMINT', 'DAYLTMINF', 'DAYLTMINS', 'DAYMVBOUTMINT', 'DAYMVBOUTMINF', 'DAYMVBOUTMINS', 'DAYVBOUTMINT', 'DAYVBOUTMINF', 'DAYVBOUTMINS'],

# Columns with only floats, missing, and NA values
'float': ['WEARHR'],

# Columns with only strings, missing, and NA values
'cat': ['PAWEEKDAY'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## acceldatabymin

In [ ]:
prefix = 'acceldatabymin'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)

data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only unsigned ints, missing, and NA values
'unsigned': ['PASTUDYDAY', 'MINSEQUENCE', 'SUSPECTMINUTE', 'MINCNT', 'PAMONTH'],

# Columns with only strings, missing, and NA values
'cat': ['PAWEEKDAY'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
new_df['SUSPECTMINUTE'] = new_df['SUSPECTMINUTE'].astype('bool')

TODO: SUSPECTMINUTE should be a bool

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)

if not missing_df.empty:
    print('Missing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('\nShadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## accelerometry

In [ ]:
prefix = 'accelerometry'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)

data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only unsigned ints, missing, and NA values
'unsigned': ['AMPA1', 'ANVDAYS'],

# Columns with only floats, missing, and NA values
'float': ['AAVMNT', 'AAMVBMF', 'AAMVBMT', 'AAMVMNS', 'AAVBMS', 'AAMVBMS', 'AAMVMNT', 'AACSM03', 'AAMVMNF', 'AAVMNF', 'AACNT', 'AALTMNF', 'AAMDMNF', 'AAVBMT', 'AALTMNT', 'AAMDMNT', 'AAVBMF', 'AALTMNS', 'AAMDMNS', 'AAVMNS'],

# Columns with only strings, missing, and NA values
'cat': ['APASTAT'],

}
new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)

if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## allclinical

In [ ]:
prefix = 'allclinical'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

Note the compression from 7016 variables to 1851.

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)

data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only dates, missing, and NA values
'date': ['SVDATE', 'DATE', 'EVDATE', 'PSDATE', 'SSDATE', 'FVDATE', 'ISEXMDT'],

# Columns with only unsigned ints, missing, and NA values
'unsigned': ['KPNR12M', 'KPNL12M', 'TMJE30D', 'TMJE30A', 'TMJF30D', 'TMJF30A', 'BPTOT', 'BPDAYCV', 'BPBEDCV', 'KPACDCV', 'INJR1', 'INJR2', 'INJR3', 'KRSRA', 'ARTR1', 'ARTR2', 'ARTR3', 'MENR1', 'MENR2', 'MENR3', 'LRR1', 'LRR2', 'OTSR1', 'OTSR2', 'OTSR3', 'INJL1', 'INJL2', 'INJL3', 'KRSLA', 'ARTL1', 'ARTL2', 'ARTL3', 'MENL1', 'MENL2', 'MENL3', 'LRL1', 'OTSL1', 'OTSL2', 'OTSL3', 'OV1AGE', 'OV2AGE', 'HYSAGE', 'BLDHRS1', 'BLDHRS2', 'BLSURD1', 'BLSURD2', 'HOURSP1', 'HOURSP2', 'PDATE1', 'PDATE2', 'PLAQHR1', 'PLAQHR2', 'SEAQHR1', 'SEAQHR2', 'UCDATE1', 'UCDATE2', 'URINHR1', 'URINHR2', 'URSURD1', 'URSURD2', 'WOMSTFR', 'WOMSTFL', 'HIPFXAG', 'SPNFXAG', 'SMKAGE', 'SMKAVE', 'SMKAMT', 'SMKSTOP', 'PIPEAGE', 'PIPEAMT', 'PIPSTOP', 'BISPYRS', 'RX30NUM', 'COMORB', 'SMKPKYR', 'PSMKYR', 'CESD', 'NWARNS', 'NNOSERV', 'NSKIP', 'NERRORS', 'BPSYS', 'BPDIAS', 'RPAVG', 'STEPST1', 'STEPST2', 'HRB4WLK', 'NUMSTOP', 'HR400WK', 'LLWGT', 'RLWGT', '400MTR', 'AGE', 'HOURWK', 'MISSWK', 'PASE', 'WEEKWK', 'WKHR7CV', 'BLUPMN1', 'BLUPMN2', 'PRRDDYS', 'URUPMN2', 'VISDYS', 'AMPA1', 'ANVDAYS', 'SF12BP', 'SF12PF', 'SF12VT', 'SF12SF', 'WTLSYR', 'AC1AR1', 'AC2AR1', 'AC3AR1', 'AC1AR2', 'AC2AR2', 'AC3AR2', 'AC1AR3', 'AC2AR3', 'AC3AR3', 'AC1AR4', 'AC2AR4', 'AC3AR4'],

# Columns with only signed ints, missing, and NA values
'signed': ['RKFHDEG', 'LKFHDEG', 'RKALNMT', 'LKALNMT', 'DFBCOLL', 'DFUCOLL', 'URUPMN1'],

# Columns with only floats, missing, and NA values
'float': ['HEIGHT', 'WEIGHT', 'BMI', 'HRSUC1', 'HRSUC2', 'HSPSS', 'HSMSS', 'WOMKPR', 'KOOSKPR', 'KOOSYMR', 'WOMADLR', 'WOMKPL', 'KOOSKPL', 'KOOSYML', 'WOMADLL', 'KOOSFSR', 'KOOSQOL', 'WOMTSL', 'WOMTSR', 'HT25MM', 'WT25KG', 'WTMAXKG', 'WTMINKG', 'DTDFIB', 'SUPVITD', 'FIBVGFR', 'SUPB12', 'DTCAFFN', 'SRVFAT', 'DTAIU', 'DTCHOL', 'PCTCOL1', 'DTPHOS', 'DTVITC', 'DTB1', 'PCTXLS', 'SUPB2', 'PCTCOL9', 'DTVITK', 'DTRET', 'SUPVITE', 'SUPNIAC', 'DTANZN', 'DTLUT', 'BAPFAT', 'PCTCARB', 'PCTSWT', 'DTACAR', 'SUPCA', 'SRVGRN', 'SRVFRT', 'SUPFOL', 'DTBCAR', 'DTPROT', 'DTPOTA', 'DTSFAT', 'SUPVITC', 'DTOLEC', 'SUPBCAR', 'DTKCAL', 'BAPPROT', 'BAPCARB', 'SUPVITA', 'SUPB6', 'NFDSDAY', 'DTNIAC', 'FIBBEAN', 'DTNA', 'DTARE', 'DTLYC', 'DTFAT', 'PCTSMAL', 'SUPFE', 'SUPCU', 'DTB12', 'DTGEN', 'DTMETH', 'SUPZINC', 'SRVVEG', 'DTCALC', 'SUPMG', 'DTDAID', 'SRVMEAT', 'DTFE', 'FIBGRN', 'DTCYST', 'DTSF', 'PCTPROT', 'SRVDRY', 'DTB6', 'SUPB1', 'SUPSE', 'DTMG', 'PCTFAT', 'DTVITD', 'DTPROA', 'DTCARB', 'PCTALCH', 'DTFOL', 'DTLIN', 'PCTLARG', 'DTVITE', 'DTCRYP', 'PCTMEDS', 'DTRIBO', 'DTZINC', 'CSTIME1', 'CSTIME2', 'RLLGTH', 'RLBACK', 'RLARM', 'RLHORIZ', 'RLVERT', 'LLLGTH', 'LLBACK', 'LLARM', 'LLHORIZ', 'LLVERT', 'TIMET1', 'TIMET2', 'ABCIRC', 'CSPACE', '20MPACE', 'RFTLPL', 'RFTHPL', 'RFTLRL', 'RFTHRL', 'RETLPL', 'RETHPL', 'RETLRL', 'RETHRL', 'LFTLPL', 'LFTHPL', 'LFTLRL', 'LFTHRL', 'LETLPL', 'LETHPL', 'LETLRL', 'LETHRL', '400MTIM', 'RFSFR', 'LESFR', 'LFSFP', 'RESFP', 'RESFR', 'LFSFR', 'RFSFP', 'LESFP', 'ICPTSKL', 'ICPTSKR', 'IPSKL', 'CPSKR', 'IPSKR', 'CPSKL', 'LLDILSM', 'LLDIFST', 'LLDILST', 'LLDIFSS', 'LLDILSI', 'LLDIFSP', 'AAVMNT', 'AAMVBMF', 'AAMVBMT', 'AAMVMNS', 'AAVBMS', 'AAMVBMS', 'AAMVMNT', 'AACSM03', 'AAMVMNF', 'AAVMNF', 'AACNT', 'AALTMNF', 'AAMDMNF', 'AAVBMT', 'AALTMNT', 'AAMDMNT', 'AAVBMF', 'AALTMNS', 'AAMDMNS', 'AAVMNS', 'IPSHL', 'IPSHR', 'ICPTSHR', 'CPSHL', 'ICPTSHL', 'CPSHR', 'SF12RP', 'SF12RE', 'SF12GH', 'SF12MH'],

# Columns with only strings, missing, and NA values
'cat': ['LRR3', 'LRL2', 'LRL3', 'STFID2', 'STFID1', 'HESTFID', 'SVXRRID', 'BPSTFID', 'RPSTFID', 'ACSTFID', 'SCSTFID', 'RCSTFID', 'W2STFID', 'W4STFID', 'K1STFID', 'ISSTFID', 'WPSTFID', 'K5STFID', 'KPSTFID', 'APASTAT', 'HVSTFID', 'WLC1IL'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)

if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## biomarkers

In [ ]:
prefix = 'biomarkers'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only unsigned ints, missing, and NA values
'unsigned': ['BLDHRS1', 'BLDHRS2', 'BLSURD1', 'BLSURD2', 'HOURSP1', 'HOURSP2', 'PDATE1', 'PDATE2', 'PLAQHR1', 'PLAQHR2', 'SEAQHR1', 'SEAQHR2', 'UCDATE1', 'UCDATE2', 'URINHR1', 'URINHR2', 'URSURD1', 'URSURD2', 'BLUPMN1', 'BLUPMN2', 'PRRDDYS', 'URUPMN2'],

# Columns with only signed ints, missing, and NA values
'signed': ['DFBCOLL', 'DFUCOLL', 'URUPMN1'],

# Columns with only floats, missing, and NA values
'float': ['HRSUC1', 'HRSUC2'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## biospec_fnih_joco_assays
TODO: Not handled yet as column naming format doesn't use visit prefixes

In [ ]:
prefix = 'biospec_fnih_joco_assays'
column_uniformity_check(prefix)

## biospec_fnih_labcorp

In [ ]:
prefix = 'biospec_fnih_labcorp'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only unsigned ints, missing, and NA values
'unsigned': ['SERUM_C2C_NUM', 'SERUM_CPII_NUM', 'SERUM_PIIANP_NUM', 'SERUM_CS846_NUM', 'SERUM_COMP_NUM', 'SERUM_HA_NUM', 'SERUM_NTXI_NUM', 'URINE_C2C_NUM', 'URINE_NTXI_NUM', 'SERUM_C2C_ALTNUM', 'SERUM_CPII_ALTNUM', 'SERUM_CS846_ALTNUM', 'SERUM_COMP_ALTNUM', 'SERUM_HA_ALTNUM', 'SERUM_NTXI_ALTNUM', 'SERUM_PIIANP_ALTNUM', 'URINE_C2C_ALTNUM', 'URINE_NTXI_ALTNUM', 'SERUM_C1_2C_LOWLIM', 'SERUM_C2C_LOWLIM', 'SERUM_COLL2_1_NO2_LOWLIM', 'SERUM_CPII_LOWLIM', 'SERUM_CS846_LOWLIM', 'SERUM_CTXI_LOWLIM', 'SERUM_COMP_LOWLIM', 'SERUM_HA_LOWLIM', 'SERUM_MMP_3_LOWLIM', 'SERUM_NTXI_LOWLIM', 'SERUM_PIIANP_LOWLIM', 'URINE_CTXII_LOWLIM', 'URINE_C1_2C_LOWLIM', 'URINE_C2C_LOWLIM', 'URINE_NTXI_LOWLIM', 'URINE_ALPHA_LOWLIM', 'URINE_BETA_LOWLIM', 'URINE_COL21N2_LOWLIM'],

# Columns with only floats, missing, and NA values
'float': ['SERUM_C1_2C_NUM', 'SERUM_COLL2_1_NO2_NUM', 'SERUM_CTXI_NUM', 'SERUM_MMP_3_NUM', 'URINE_CTXII_NUM', 'URINE_C1_2C_NUM', 'URINE_CREATININE_NUM', 'URINE_ALPHA_NUM', 'URINE_BETA_NUM', 'URINE_COL21N2_NUM', 'URINE_CTXII_NUMCA', 'URINE_C1_2C_NUMCA', 'URINE_C2C_NUMCA', 'URINE_NTXI_NUMCA', 'URINE_ALPHA_NUMCA', 'URINE_BETA_NUMCA', 'URINE_COL21N2_NUMCA', 'SERUM_C1_2C_ALTNUM', 'SERUM_COLL2_1_NO2_ALTNUM', 'SERUM_CTXI_ALTNUM', 'SERUM_MMP_3_ALTNUM', 'URINE_CTXII_ALTNUM', 'URINE_C1_2C_ALTNUM', 'URINE_ALPHA_ALTNUM', 'URINE_BETA_ALTNUM', 'URINE_COL21N2_ALTNUM', 'URINE_CTXII_ALTNUMCA', 'URINE_C1_2C_ALTNUMCA', 'URINE_C2C_ALTNUMCA', 'URINE_NTXI_ALTNUMCA', 'URINE_ALPHA_ALTNUMCA', 'URINE_BETA_ALTNUMCA', 'URINE_COL21N2_ALTNUMCA', 'URINE_COL21N2SD', 'URINE_COL21N2CV'],

# Columns with only strings, missing, and NA values
'cat': ['SERUM_C1_2C_LC', 'SERUM_C2C_LC', 'SERUM_COLL2_1_NO2_LC', 'SERUM_CPII_LC', 'SERUM_CS846_LC', 'SERUM_CTXI_LC', 'SERUM_COMP_LC', 'SERUM_HA_LC', 'SERUM_MMP_3_LC', 'SERUM_NTXI_LC', 'SERUM_PIIANP_LC', 'URINE_CTXII_LC', 'URINE_C1_2C_LC', 'URINE_C2C_LC', 'URINE_CREATININE_LC', 'URINE_NTXI_LC', 'URINE_ALPHA_LC', 'URINE_BETA_LC', 'SERUM_C1_2C_COMMENT', 'SERUM_C2C_COMMENT', 'SERUM_COLL2_1_NO2_COMMENT', 'SERUM_CPII_COMMENT', 'SERUM_CS846_COMMENT', 'SERUM_CTXI_COMMENT', 'SERUM_COMP_COMMENT', 'SERUM_HA_COMMENT', 'SERUM_MMP_3_COMMENT', 'SERUM_NTXI_COMMENT', 'SERUM_PIIANP_COMMENT', 'URINE_CTXII_COMMENT', 'URINE_C1_2C_COMMENT', 'URINE_C2C_COMMENT', 'URINE_CREATININE_COMMENT', 'URINE_NTXI_COMMENT', 'URINE_ALPHA_COMMENT', 'URINE_BETA_COMMENT', 'SERUM_C1_2C_HQC', 'SERUM_C2C_HQC', 'SERUM_COLL2_1_NO2_HQC', 'SERUM_CPII_HQC', 'SERUM_CS846_HQC', 'SERUM_CTXI_HQC', 'SERUM_COMP_HQC', 'SERUM_HA_HQC', 'SERUM_MMP_3_HQC', 'SERUM_NTXI_HQC', 'SERUM_PIIANP_HQC', 'URINE_CTXII_HQC', 'URINE_CREATININE_HQC', 'URINE_NTXI_HQC', 'URINE_ALPHA_HQC', 'URINE_BETA_HQC', 'SERUM_C1_2C_LQC', 'SERUM_C2C_LQC', 'SERUM_COLL2_1_NO2_LQC', 'SERUM_CPII_LQC', 'SERUM_CS846_LQC', 'SERUM_CTXI_LQC', 'SERUM_COMP_LQC', 'SERUM_HA_LQC', 'SERUM_MMP_3_LQC', 'SERUM_NTXI_LQC', 'SERUM_PIIANP_LQC', 'URINE_CTXII_LQC', 'URINE_C1__2C_LQC', 'URINE_C2C_LQC', 'URINE_CREATININE_LQC', 'URINE_NTXI_LQC', 'URINE_ALPHA_LQC', 'URINE_BETA_LQC', 'SERUM_C1_2C_MQC', 'SERUM_C2C_MQC', 'SERUM_CPII_MQC', 'SERUM_CS846_MQC', 'SERUM_MMP_3_MQC', 'URINE_C1__2C_MQC', 'URINE_C2C_MQC', 'SERUM_C1_2C_KIT_LOT_NUM', 'SERUM_C2C_KIT_LOT_NUM', 'SERUM_COLL2_1_NO2_KIT_LOT_NUM', 'SERUM_CPII_KIT_LOT_NUM', 'SERUM_CS846_KIT_LOT_NUM', 'SERUM_CTXI_KIT_LOT_NUM', 'SERUM_COMP_KIT_LOT_NUM', 'SERUM_HA_KIT_LOT_NUM', 'SERUM_MMP_3_KIT_LOT_NUM', 'SERUM_NTXI_KIT_LOT_NUM', 'SERUM_PIIANP_KIT_LOT_NUM', 'URINE_CTXII_KIT_LOT_NUM', 'URINE_C1__2C_KIT_LOT_NUM', 'URINE_C2C_KIT_LOT_NUM', 'URINE_CREATININE_KIT_LOT_NUM', 'URINE_NTXI_KIT_LOT_NUM', 'URINE_ALPHA_KIT_LOT_NUM', 'URINE_BETA_KIT_LOT_NUM', 'SERUM_C1_2C_PLATE_ID', 'SERUM_C2C_PLATE_ID', 'SERUM_COLL2_1_NO2_PLATE_ID', 'SERUM_CPII_PLATE_ID', 'SERUM_CS846_PLATE_ID', 'SERUM_CTXI_PLATE_ID', 'SERUM_COMP_PLATE_ID', 'SERUM_HA_PLATE_ID', 'SERUM_MMP_3_PLATE_ID', 'SERUM_NTXI_PLATE_ID', 'SERUM_PIIANP_PLATE_ID', 'URINE_CTXII_PLATE_ID', 'URINE_C1__2C_PLATE_ID', 'URINE_C2C_PLATE_ID', 'URINE_CREATININE_PLATE_ID', 'URINE_NTXI_PLATE_ID', 'URINE_ALPHA_PLATE_ID', 'URINE_BETA_PLATE_ID'],

}


new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## boneancillarystudy

In [ ]:
prefix = 'boneancillarystudy'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only dates, missing, and NA values
'date': ['KNEEDXADT', 'TRABMRSEQDT', 'HIPDXADT', 'PDATE'],

# Columns with only unsigned ints, missing, and NA values
'unsigned': ['KNEEDXASFWARE', 'VITD'],

# Columns with only floats, missing, and NA values
'float': ['MEDIALBMD', 'LATERALBMD', 'BMDRATIO', 'BVF', 'TRN', 'TRSP', 'TRTH', 'NECKBMD'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## enrollees

In [ ]:
prefix = 'enrollees'
tmp_df = create_df(prefix)
print(tmp_df.shape)
print(tmp_df.columns)

In [ ]:
# Are all variables associated with the same release VERSION?
tmp_df['VERSION'].value_counts()

From reading about the data, the enrollees data is best stored as two separate dataframes:
* One has basic data about the enrollees collected at either the IEI or EV. 
* The second tracks participation in different image groups by visit.

### Enrollee Data

In [ ]:
enrollee_df = tmp_df[tmp_df['Visit'] == 'P02'].copy()
enrollee_df.dropna(how='all', axis='columns', inplace=True)  # Drop all columns for data that wasn't collected during visit P02
enrollee_df.drop('Visit', axis='columns', inplace=True)  # this information is independent of the visit

enrollee_df = enrollee_df.join(tmp_df[tmp_df['Visit'] == 'V00'][['ID', 'CHRTHLF', 'COHORT', 'SITE']].set_index('ID'), on='ID')
enrollee_df = enrollee_df.join(tmp_df[tmp_df['Visit'] == 'V01'][['ID', 'HADINTV']].set_index('ID'), on='ID')
enrollee_df = enrollee_df.astype({'SITE': 'category'})
enrollee_df.set_index('ID', inplace=True)

In [ ]:
sanity_check(enrollee_df)
print()
print(enrollee_df.dtypes)

In [ ]:
pickle.dump(enrollee_df, open('pkl/enrollees_values.pkl', 'wb'))
enrollee_df = None

### Image Group Participation

In [ ]:
df_list = []
for v in ['V00', 'V01', 'V02', 'V03', 'V04', 'V05', 'V06', 'V08', 'V10']:
    image_groups_df = tmp_df[tmp_df['Visit'] == v].copy()
    image_groups_df.dropna(how='all', axis='columns', inplace=True)
    if v == 'V00':
        image_groups_df.drop(['CHRTHLF', 'COHORT', 'SITE'], axis='columns', inplace=True)
    if v == 'V01':
        image_groups_df.drop(['HADINTV'], axis='columns', inplace=True)
    df_list.append(image_groups_df)
image_groups_df = pd.concat(df_list, axis=0)
image_groups_df = image_groups_df.set_index(['ID', 'Visit'])

In [ ]:
sanity_check(image_groups_df)
print()
print(image_groups_df.dtypes)

In [ ]:
pickle.dump(image_groups_df, open('pkl/image_groups_values.pkl', 'wb'))
df_list = None
image_groups_df = None

## flxr_kneealign_cooke

In [ ]:
prefix = 'flxr_kneealign_cooke'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only floats, missing, and NA values
'float': ['HKANGLE'],

# Columns with only strings, missing, and NA values
'cat': ['BARCDDC'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))
    

## flxr_kneealign_duryea

In [ ]:
prefix = 'flxr_kneealign_duryea'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only floats, missing, and NA values
'float': ['HKANGJD', 'FEMLEN', 'TIBLEN', 'APPLL'],

# Columns with only strings, missing, and NA values
'cat': ['BRCDHJD'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## kmri_fnih_boneshape_imorphics

In [ ]:
prefix = 'kmri_fnih_boneshape_imorphics'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only floats, missing, and NA values
'float': ['MF_TAB', 'LF_TAB', 'MT_TAB', 'LT_TAB', 'MP_TAB', 'LP_TAB', 'NOTCH', 'TRFLAT', 'TRFMED', 'NFEMUROAVECTOR', 'NTIBIAOAVECTOR', 'NPATELLAOAVECTOR'],

# Columns with only strings, missing, and NA values
'cat': ['BARCDIM'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## kmri_fnih_qcart_Chondrometrics

In [ ]:
prefix = 'kmri_fnih_qcart_Chondrometrics'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only floats, missing, and NA values
'float': ['WMTVCL', 'WMTSBA', 'WMTVCN', 'WMTMTH', 'WMTACS', 'WMTPD', 'WMTCAAB', 'WMTMTC', 'WMTMAV', 'WMTCTS', 'WMTACV', 'CMTMAT', 'CMTMTH', 'EMTMTH', 'IMTMTH', 'AMTMTH', 'PMTMTH', 'CMTPD', 'EMTPD', 'IMTPD', 'AMTPD', 'PMTPD', 'BMFVCL', 'BMFSBA', 'BMFVCN', 'BMFMTH', 'BMFACS', 'BMFPD', 'BMFCAAB', 'BMFMTC', 'BMFMAV', 'BMFCTS', 'BMFACV', 'CBMFMAT', 'CBMFMTH', 'EBMFMTH', 'IBMFMTH', 'CBMFPD', 'EBMFPD', 'IBMFPD', 'WMTFVCL', 'WMTFVCN', 'WMTFMTH', 'WMTFMAV', 'BMTFMAT', 'BMTFMTH', 'WLTVCL', 'WLTSBA', 'WLTVCN', 'WLTMTH', 'WLTACS', 'WLTPD', 'WLTCAAB', 'WLTMTC', 'WLTMAV', 'WLTCTS', 'WLTACV', 'CLTMAT', 'CLTMTH', 'ELTMTH', 'ILTMTH', 'ALTMTH', 'PLTMTH', 'CLTPD', 'ELTPD', 'ILTPD', 'ALTPD', 'PLTPD', 'BLFVCL', 'BLFSBA', 'BLFVCN', 'BLFMTH', 'BLFACS', 'BLFPD', 'BLFCAAB', 'BLFMTC', 'BLFMAV', 'BLFCTS', 'BLFACV', 'CBLFMAT', 'CBLFMT', 'EBLFMT', 'IBLFMT', 'CBLFPD', 'EBLFPD', 'IBLFPD', 'WLTFVCL', 'WLTFVCN', 'WLTFMTH', 'WLTFMAV', 'BLTFMAT', 'BLTFMTH'],

# Columns with only strings, missing, and NA values
'cat': ['BARCDFE'],

}


new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## kmri_fnih_qcart_biomediq

In [ ]:
prefix = 'kmri_fnih_qcart_biomediq'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only floats, missing, and NA values
'float': ['MEDIALTIBIALCARTILAGE', 'LATERALTIBIALCARTILAGE', 'MEDIALFEMORALCARTILAGE', 'LATERALFEMORALCARTILAGE', 'PATELLARCARTILAGE', 'MEDIALMENISCUS', 'LATERALMENISCUS'],

# Columns with only strings, missing, and NA values
'cat': ['BARCDED'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

##  kmri_fnih_sbp_qmetrics

In [ ]:
prefix = 'kmri_fnih_sbp_qmetrics'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only floats, missing, and NA values
'float': ['SUBBAREA_MEDFEM', 'CURAVERAGE_SUBB_MEDFEM', 'CURSD_SUBB_MEDFEM', 'CUR_5PC_SUBB_MEDFEM', 'CUR_95PC_SUBB_MEDFEM', 'SCRAVERAGE_SUBB_MEDFEM', 'SCRSD_SUBB_MEDFEM', 'SCR_5PC_SUBB_MEDFEM', 'SCR_95PC_SUBB_MEDFEM', 'SUBBAREA_LATFEM', 'CURAVERAGE_SUBB_LATFEM', 'CURSD_SUBB_LATFEM', 'CUR_5PC_SUBB_LATFEM', 'CUR_95PC_SUBB_LATFEM', 'SCRAVERAGE_SUBB_LATFEM', 'SCRSD_SUBB_LATFEM', 'SCR_5PC_SUBB_LATFEM', 'SCR_95PC_SUBB_LATFEM', 'SUBBAREA_MEDTIB', 'CURAVERAGE_SUBB_MEDTIB', 'CURSD_SUBB_MEDTIB', 'CUR_5PC_SUBB_MEDTIB', 'CUR_95PC_SUBB_MEDTIB', 'SCRAVERAGE_SUBB_MEDTIB', 'SCRSD_SUBB_MEDTIB', 'SCR_5PC_SUBB_MEDTIB', 'SCR_95PC_SUBB_MEDTIB', 'SUBBAREA_LATTIB', 'CURAVERAGE_SUBB_LATTIB', 'CURSD_SUBB_LATTIB', 'CUR_5PC_SUBB_LATTIB', 'CUR_95PC_SUBB_LATTIB', 'SCRAVERAGE_SUBB_LATTIB', 'SCRSD_SUBB_LATTIB', 'SCR_5PC_SUBB_LATTIB', 'SCR_95PC_SUBB_LATTIB'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## kmri_fnih_sq_moaks_bicl

In [ ]:
prefix = 'kmri_fnih_sq_moaks_bicl'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only unsigned ints, missing, and NA values
'unsigned': ['MBMNFMA', 'MBMNFLA', 'MBMNFMC', 'MBMNFLC', 'MBMNFMP', 'MBMNFLP', 'MBMNSS', 'MBMNTMA', 'MBMNTLA', 'MBMNTMC', 'MBMNTLC', 'MBMNTMP', 'MBMNTLP', 'MBMNPM', 'MBMNPL'],

# Columns with only strings, missing, and NA values
'cat': ['READER', 'MCMNTS', 'MTCMNTS'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## kmri_poma_incoa_moaks_bicl
TODO: Not handled yet as column naming format doesn't use visit prefixes

In [ ]:
prefix = 'kmri_poma_incoa_moaks_bicl'
column_uniformity_check(prefix)

## kmri_poma_tkr_chondrometrics
TODO: Not handled yet as column naming format doesn't use visit prefixes

In [ ]:
prefix = 'kmri_poma_tkr_chondrometrics'
column_uniformity_check(prefix)

## kmri_poma_tkr_moaks_bicl
TODO: Not handled yet as column naming format doesn't use visit prefixes

In [ ]:
prefix = 'kmri_poma_tkr_moaks_bicl'
column_uniformity_check(prefix)

## kmri_qcart_eckstein

In [ ]:
prefix = 'kmri_qcart_eckstein'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only floats, missing, and NA values
'float': ['CBLFMAT', 'CBMFPD', 'BMFACV', 'BMTFMTH', 'WLTFMAV', 'IMTMTH', 'PLTMTH', 'BLFVCL', 'WMTCTS', 'BMFPD', 'BMFMTC', 'EMTMTH', 'WLTVCL', 'BLFACS', 'WMTVCN', 'WMTMAV', 'EBLFPD', 'AMTPD', 'WLTFMTH', 'WMTMTH', 'EMTPD', 'WLTFVCL', 'CMTMTH', 'CLTMAT', 'ILTMTH', 'WMTCAAB', 'BMFVCL', 'PMTPD', 'BLFACV', 'BMTFMAT', 'WMTFMTH', 'IBLFPD', 'EBMFPD', 'BMFACS', 'CMTPD', 'BLTFMAT', 'WLTCTS', 'WLTMAV', 'IBMFMTH', 'WMTFMAV', 'AMTMTH', 'WMTFVCN', 'ALTPD', 'WMTSBA', 'BMFVCN', 'BLFMTC', 'ELTMTH', 'PLTPD', 'BLFMAV', 'IMTPD', 'WLTFVCN', 'BLFVCN', 'EBMFMTH', 'WLTACV', 'IBLFMT', 'ELTPD', 'BLFCAAB', 'WMTFVCL', 'WLTMTH', 'CMTMAT', 'WLTCAAB', 'BLFSBA', 'IBMFPD', 'BLFCTS', 'CLTMTH', 'WLTMTC', 'CLTPD', 'CBMFMTH', 'WMTPD', 'PMTMTH', 'BLFMTH', 'WMTACS', 'ILTPD', 'CBMFMAT', 'EBLFMT', 'WLTSBA', 'WMTACV', 'ALTMTH', 'CBLFPD', 'BLTFMTH', 'BMFCAAB', 'CBLFMT', 'WLTACS', 'BMFMTH', 'BMFCTS', 'BMFSBA', 'BMFMAV', 'BLFPD', 'WMTMTC', 'WLTVCN', 'WMTVCL', 'WLTPD'],

# Columns with only strings, missing, and NA values
'cat': ['BARCDFE'],

}
new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## kmri_qcart_link

In [ ]:
prefix = 'kmri_qcart_link'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only floats, missing, and NA values
'float': ['LFT2AV', 'LTT2AV', 'MFT2AV', 'MTT2AV', 'PATT2AV'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## kmri_qcart_vs

In [ ]:
prefix = 'kmri_qcart_vs'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only unsigned ints, missing, and NA values
'unsigned': ['PMFSADA'],

# Columns with only floats, missing, and NA values
'float': ['LTWVOL', 'PATTMAX', 'CLFTAVG', 'TRFNVOL', 'WFTAVG', 'CMFSABC', 'CMTSABC', 'PATBMEV', 'CLFTVAR', 'CLTSAAS', 'LTNVOL', 'LTSABC', 'TRFTMAX', 'PLFSAAS', 'LTRBMEV', 'LTTMAX', 'CMTTAVG', 'CLTSABC', 'PLFSADA', 'MTSABC', 'LPBMEV', 'MTNVOL', 'CLTSADA', 'CMTCAVG', 'MTSADA', 'CMFSAAS', 'PMFBMEV', 'CMTRENG', 'MTWVOL', 'PLFSABC', 'CLFWVOL', 'WFTVAR', 'CMFTVAR', 'CMTSADA', 'CMTSAAS', 'CLTRENG', 'CLFCAVG', 'CMFSADA', 'CLFRENG', 'CLTCAVG', 'CMTNVOL', 'WFTMAX', 'CMFTAVG', 'LTSAAS', 'CMTWVOL', 'CLFTMAX', 'MPBMEV', 'CMFCAVG', 'LTRENG', 'CLFSADA', 'CMFRENG', 'CLFSAAS', 'CMFWVOL', 'PLFTAVG', 'CLFSABC', 'MTRBMEV', 'CMTTVAR', 'LTTAVG', 'CLTTAVG', 'MTBMEV', 'PATCAVG', 'PLFRENG', 'CLFNVOL', 'CMTTMAX', 'MTTAVG', 'PMFTVAR', 'PLFWVOL', 'CLTNVOL', 'CLTWVOL', 'LTCAVG', 'PMFTAVG', 'PLFCAVG', 'LTBMEV', 'PMFCAVG', 'PLFTVAR', 'WFSADA', 'PATTAVG', 'MTCAVG', 'CLTTVAR', 'CMFNVOL', 'CLFBMEV', 'WFWVOL', 'CMFTMAX', 'TRFTAVG', 'WFSABC', 'MTRENG', 'MTSAAS', 'PATRENG', 'PMFWVOL', 'PLFNVOL', 'LTSADA', 'PMFSABC', 'MTTMAX', 'WFCAVG', 'TRFRENG', 'PATSAAS', 'CLTTMAX', 'MTTVAR', 'TRFWVOL', 'PMFRENG', 'PATNVOL', 'PLFTMAX', 'TRFCAVG', 'TRFSAAS', 'WFSAAS', 'PATWVOL', 'WFRENG', 'TRFTVAR', 'PLFBMEV', 'WFNVOL', 'PATSABC', 'PATTVAR', 'TRFSABC', 'TRFSADA', 'PMFSAAS', 'PMFNVOL', 'PMFTMAX', 'LTTVAR', 'CMFBMEV', 'PATSADA'],

# Columns with only strings, missing, and NA values
'cat': ['BARCDVS'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## kmri_sq_bicl

In [ ]:
prefix = 'kmri_sq_bicl'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only unsigned ints, missing, and NA values
'unsigned': ['BNBMLS', 'BNBMLSK'],

# Columns with only strings, missing, and NA values
'cat': ['READER'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## kmri_sq_blksbml_bicl

In [ ]:
prefix = 'kmri_sq_blksbml_bicl'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only unsigned ints, missing, and NA values
'unsigned': ['BBMLNUM'],

}


new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## kmri_sq_moaks_bicl

In [ ]:
prefix = 'kmri_sq_moaks_bicl'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {}
new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## kmri_sq_worms_link

In [ ]:
prefix = 'kmri_sq_worms_link'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only unsigned ints, missing, and NA values
'unsigned': ['MWPOPCY'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## kxr_fnih_bti_duke

In [ ]:
prefix = 'kxr_fnih_bti_duke'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only floats, missing, and NA values
'float': ['BTI_H0', 'BTI_H1', 'BTI_H2', 'BTI_V0', 'BTI_V1', 'BTI_V2'],

# Columns with only strings, missing, and NA values
'cat': ['BARCDBTI'],

}


new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## kxr_fta_duryea

In [ ]:
prefix = 'kxr_fta_duryea'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only floats, missing, and NA values
'float': ['FTANGLE'],

# Columns with only strings, missing, and NA values
'cat': ['BRCDJD'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## kxr_qjsw_duryea

In [ ]:
prefix = 'kxr_qjsw_duryea'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only floats, missing, and NA values
'float': ['CFWDTH', 'MCMJSW', 'JSW175', 'JSW200', 'JSW250', 'JSW300', 'JSW225', 'TPCFDS', 'BMANG', 'JSW150', 'JSW275', 'LJSW850', 'LJSW900', 'LJSW700', 'LJSW825', 'LJSW750', 'LJSW875', 'LJSW725', 'LJSW775', 'LJSW800', 'XMJSW', 'IMPIXSZ'],

# Columns with only strings, missing, and NA values
'cat': ['BARCDJD'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## kxr_qjsw_rel_duryea

In [ ]:
prefix = 'kxr_qjsw_rel_duryea'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only floats, missing, and NA values
'float': ['CFWDTH', 'MCMJSW', 'JSW175', 'JSW200', 'JSW250', 'JSW300', 'JSW225', 'TPCFDS', 'BMANG', 'JSW150', 'JSW275', 'LJSW850', 'LJSW900', 'LJSW700', 'LJSW825', 'LJSW750', 'LJSW875', 'LJSW725', 'LJSW775', 'LJSW800', 'XMJSW', 'IMPIXSZ'],

# Columns with only strings, missing, and NA values
'cat': ['BARCDJD'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## kxr_sq_bu

In [ ]:
prefix = 'kxr_sq_bu'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only floats, missing, and NA values
'float': ['XRJSM', 'XRJSL'],

# Columns with only strings, missing, and NA values
'cat': ['BARCDBU'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## kxr_sq_rel_bu

In [ ]:
prefix = 'kxr_sq_rel_bu'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only floats, missing, and NA values
'float': ['XRJSM', 'XRJSL'],

# Columns with only strings, missing, and NA values
'cat': ['BARCDBU'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## measinventory

In [ ]:
prefix = 'measinventory'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only unsigned ints, missing, and NA values
'unsigned': ['AGE'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## MIF

In [ ]:
prefix = 'mif'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

It isn't clear what to do with INGCODE, they seem to be 9 digits plus '.0' except for some values labelled 'M'

In [ ]:
data_stats_summary(tmp_df, data_stats_df)

data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only unsigned ints, missing, and NA values
'unsigned': ['INGCODE'],

# Columns with only strings, missing, and NA values
'cat': ['MIFNAME', 'INGNAME'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)

if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## MRI

In [ ]:
prefix = 'mri'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)

data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only dates, missing, and NA values
'date': ['MRDATE'],

# Columns with only unsigned ints, missing, and NA values
'unsigned': ['MRSURDY'],

# Columns with only strings, missing, and NA values
'cat': ['MEXAMTP', 'MRBARCD', 'MRTECID', 'MQCCMNT'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)

if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## outcomes

In [ ]:
prefix = 'outcomes'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only dates, missing, and NA values
'date': ['ERKDATE', 'ELKDATE', 'ERHDATE', 'ELHDATE', 'EDDDATE'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## sageancillarystudy

In [ ]:
prefix = 'sageancillarystudy'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only dates, missing, and NA values
'date': ['SAGEDATE'],

# Columns with only unsigned ints, missing, and NA values
'unsigned': ['SAGEAGE', 'SAGEGENDER', 'SAGEDM', 'SAGEDMMEDS', 'SAGEDMMOUTH', 'SAGEDMINJ', 'SAGEARM', 'SAGENOARM', 'SAGEEQFAIL', 'SAGECLOTH'],

# Columns with only floats, missing, and NA values
'float': ['SAGE'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))

## subjectchar

The data that was held in SubjectChar files has been added to allclinical. Why subjectchar00 is still shipped is a mystery.

In [ ]:
# Grab the variable names from both subjectchar000 and allclinical00
tmp_df, _ = pyreadstat.read_file_multiprocessing(pyreadstat.read_sas7bdat, data_dir + 'subjectchar00.sas7bdat',
                                                            num_processes=6, user_missing=True)
sc_vars = set(tmp_df.columns)
tmp_df, _ = pyreadstat.read_file_multiprocessing(pyreadstat.read_sas7bdat, data_dir + 'allclinical00.sas7bdat',
                                                            num_processes=6, user_missing=True)
ac_vars = set(tmp_df.columns)

# Display unique variables that are in subjectchar00 that aren't included in allclinical00
print(sc_vars - ac_vars)

Nothing. Seems like SubjectChar00 is included for no reason.

## xray

In [ ]:
prefix = 'xray'
column_uniformity_check(prefix)

In [ ]:
tmp_df = create_df(prefix)
print(tmp_df.shape)
print('\nStarting dataframe size: {:.2f}MB'.format(tmp_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
data_stats_df, done_df = gather_column_data_stats(tmp_df)

In [ ]:
data_stats_summary(tmp_df, data_stats_df)
data_stats_df

In [ ]:
suggest_conversions(data_stats_df)

In [ ]:
targets = {
# Columns with only dates, missing, and NA values
'date': ['XRDATE'],

# Columns with only strings, missing, and NA values
'cat': ['EXAMTP', 'XRBARCD', 'XRTECID'],

}

new_df, missing_df = convert_columns(targets, data_stats_df, tmp_df)

In [ ]:
sanity_check(new_df)
print()
print(new_df.dtypes)
if not missing_df.empty:
    print('\nMissing values present, shadow dataframe created.')
    print(missing_df)

In [ ]:
print('\nFinal dataframe size: {:.2f}MB'.format(new_df.memory_usage(deep=True).sum() / (1024**2)))
print('Shadow dataframe size: {:.2f}MB'.format(missing_df.memory_usage(deep=True).sum() / (1024**2)))

In [ ]:
pickle.dump(new_df, open('pkl/' + prefix + '_values.pkl', 'wb'))
if not missing_df.empty:
    pickle.dump(missing_df, open('pkl/' + prefix + '_missing_values.pkl', 'wb'))